In [17]:
#import itertools


class Sentence():

    def evaluate(self, model):
        """Evaluates the logical sentence."""
        raise Exception("nothing to evaluate")

    def formula(self):
        """Returns string formula representing logical sentence."""
        return ""

    def symbols(self):
        """Returns a set of all symbols in the logical sentence."""
        return set()

    @classmethod
    def validate(cls, sentence):
        if not isinstance(sentence, Sentence):
            raise TypeError("must be a logical sentence")

    @classmethod
    def parenthesize(cls, s):
        """Parenthesizes an expression if not already parenthesized."""
        def balanced(s):
            """Checks if a string has balanced parentheses."""
            count = 0
            for c in s:
                if c == "(":
                    count += 1
                elif c == ")":
                    if count <= 0:
                        return False
                    count -= 1
            return count == 0
        if not len(s) or s.isalpha() or (
            s[0] == "(" and s[-1] == ")" and balanced(s[1:-1])
        ):
            return s
        else:
            return f"({s})"


class Symbol():

    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return isinstance(other, Symbol) and self.name == other.name

    def __hash__(self):
        return hash(("symbol", self.name))

    def __repr__(self):
        return self.name

    def evaluate(self, model):
        try:
            return bool(model[self.name])
        except KeyError:
            raise EvaluationException(f"variable {self.name} not in model")

    def formula(self):
        return self.name

    def symbols(self):
        return {self.name}


class Not():
    def __init__(self, operand):
        #Sentence.validate(operand)
        self.operand = operand

    def __eq__(self, other):
        return isinstance(other, Not) and self.operand == other.operand

    def __hash__(self):
        return hash(("not", hash(self.operand)))

    def __repr__(self):
        return f"Not({self.operand})"

    def evaluate(self, model):
        return not self.operand.evaluate(model)

    def formula(self):
        return "Â¬" + Sentence.parenthesize(self.operand.formula())

    def symbols(self):
        return self.operand.symbols()


class And():
    def __init__(self, *conjuncts):
        #for conjunct in conjuncts:
           # Sentence.validate(conjunct)
        self.conjuncts = list(conjuncts)

    def __eq__(self, other):
        return isinstance(other, And) and self.conjuncts == other.conjuncts

    def __hash__(self):
        return hash(
            ("and", tuple(hash(conjunct) for conjunct in self.conjuncts))
        )

    def __repr__(self):
        conjunctions = ", ".join(
            [str(conjunct) for conjunct in self.conjuncts]
        )
        return f"And({conjunctions})"

    def add(self, conjunct):
        #Sentence.validate(conjunct)
        self.conjuncts.append(conjunct)

    def evaluate(self, model):
        return all(conjunct.evaluate(model) for conjunct in self.conjuncts)

    def formula(self):
        if len(self.conjuncts) == 1:
            return self.conjuncts[0].formula()
        return " âˆ§ ".join([Sentence.parenthesize(conjunct.formula())
                           for conjunct in self.conjuncts])

    def symbols(self):
        return set.union(*[conjunct.symbols() for conjunct in self.conjuncts])


class Or():
    def __init__(self, *disjuncts):
        #for disjunct in disjuncts:
           # Sentence.validate(disjunct)
        self.disjuncts = list(disjuncts)

    def __eq__(self, other):
        return isinstance(other, Or) and self.disjuncts == other.disjuncts

    def __hash__(self):
        return hash(
            ("or", tuple(hash(disjunct) for disjunct in self.disjuncts))
        )

    def __repr__(self):
        disjuncts = ", ".join([str(disjunct) for disjunct in self.disjuncts])
        return f"Or({disjuncts})"

    def evaluate(self, model):
        return any(disjunct.evaluate(model) for disjunct in self.disjuncts)

    def formula(self):
        if len(self.disjuncts) == 1:
            return self.disjuncts[0].formula()
        return " âˆ¨  ".join([Sentence.parenthesize(disjunct.formula())
                            for disjunct in self.disjuncts])

    def symbols(self):
        return set.union(*[disjunct.symbols() for disjunct in self.disjuncts])


class Implication():
    def __init__(self, antecedent, consequent):
        #Sentence.validate(antecedent)
        #Sentence.validate(consequent)
        self.antecedent = antecedent
        self.consequent = consequent

    def __eq__(self, other):
        return (isinstance(other, Implication)
                and self.antecedent == other.antecedent
                and self.consequent == other.consequent)

    def __hash__(self):
        return hash(("implies", hash(self.antecedent), hash(self.consequent)))

    def __repr__(self):
        return f"Implication({self.antecedent}, {self.consequent})"

    def evaluate(self, model):
        return ((not self.antecedent.evaluate(model))
                or self.consequent.evaluate(model))

    def formula(self):
        antecedent = Sentence.parenthesize(self.antecedent.formula())
        consequent = Sentence.parenthesize(self.consequent.formula())
        return f"{antecedent} => {consequent}"

    def symbols(self):
        return set.union(self.antecedent.symbols(), self.consequent.symbols())


class Biconditional():
    def __init__(self, left, right):
       # Sentence.validate(left)
       # Sentence.validate(right)
        self.left = left
        self.right = right

    def __eq__(self, other):
        return (isinstance(other, Biconditional)
                and self.left == other.left
                and self.right == other.right)

    def __hash__(self):
        return hash(("biconditional", hash(self.left), hash(self.right)))

    def __repr__(self):
        return f"Biconditional({self.left}, {self.right})"

    def evaluate(self, model):
        return ((self.left.evaluate(model)
                 and self.right.evaluate(model))
                or (not self.left.evaluate(model)
                    and not self.right.evaluate(model)))

    def formula(self):
        left = Sentence.parenthesize(str(self.left))
        right = Sentence.parenthesize(str(self.right))
        return f"{left} <=> {right}"

    def symbols(self):
        return set.union(self.left.symbols(), self.right.symbols())


def model_check(knowledge, query):
    """Checks if knowledge base entails query."""

    def check_all(knowledge, query, symbols, model):
        """Checks if knowledge base entails query, given a particular model."""

        # If model has an assignment for each symbol
        if not symbols:

            # If knowledge base is true in model, then query must also be true
            if knowledge.evaluate(model):
                print("ll")
                print(model)
                return query.evaluate(model)
                print(model)
            return True
        else:

            # Choose one of the remaining unused symbols
            remaining = symbols.copy()
            p = remaining.pop()

            # Create a model where the symbol is true
            model_true = model.copy()
            model_true[p] = True

            # Create a model where the symbol is false
            model_false = model.copy()
            model_false[p] = False

            # Ensure entailment holds in both models
            return (check_all(knowledge, query, remaining, model_true) and
                    check_all(knowledge, query, remaining, model_false))

    # Get all symbols in both knowledge and query
    symbols = set.union(knowledge.symbols(), query.symbols())

    # Check that knowledge entails query
    return check_all(knowledge, query, symbols, dict())

In [2]:
rain = Symbol("rain")
hagrid = Symbol("hagrid")
dumbledore = Symbol("dumbledore")

knowledge = And(
    Implication(Not(rain), hagrid),
    Or(hagrid, dumbledore),
    Not(hagrid)
)
#Or(hagrid, dumbledore),
    #Not(And(hagrid, dumbledore)),
    #dumbledore
print(model_check(knowledge, hagrid))

ll
{'hagrid': False, 'dumbledore': True, 'rain': True}
False


In [ ]:
knowledge.conjuncts

In [ ]:
rain

In [ ]:
hash(And)

In [ ]:
knowledge.conjuncts

In [ ]:
hash(
            ("and", tuple(hash(conjunct) for conjunct in knowledge.conjuncts))
        )


In [ ]:
("and", tuple(hash(conjunct) for conjunct in knowledge.conjuncts))

In [ ]:
hash(knowledge)

In [ ]:
knowledge

In [ ]:
knowledge.symbols()

In [ ]:
rain.symbols()

In [ ]:
set.union(knowledge.symbols(),rain.symbols())

In [ ]:
set.union(*[conjunct.symbols() for conjunct in knowledge.conjuncts])

In [ ]:
knowledge.formula()

In [ ]:
symbols = set.union(knowledge.symbols(),rain.symbols())
remaining = symbols.copy()
p = remaining.pop()

In [ ]:
remaining

In [ ]:
knowledge.evaluate({'rain': True, 'dumbledore': True, 'hagrid': False})

In [ ]:
rain.evaluate({'rain': False, 'dumbledore': True, 'hagrid': True})

In [ ]:
rain.evaluate()

In [3]:
def get_nie(size):
    neighbor = [[[] for i in range(size+1)] for j in range(size+1)]
    for i in range(1,size+1):
        for j in range(1,size+1):
            if(i!=1):
                neighbor[i][j].append(f'{i-1}{j}')
            if(j!=size):
                neighbor[i][j].append(f'{i}{j+1}')
            if(i!=size):
                neighbor[i][j].append(f'{i+1}{j}')
            if(j!=1):
                neighbor[i][j].append(f'{i}{j-1}')
    return neighbor

In [4]:
neighbors = get_nie(4)

In [55]:
def join_sym(smb,terms,extra='',extra2=''):
    temp = f'{smb}('
    for i in terms:
        temp = temp +f' {extra}{i}{extra2},'
    temp = temp[:-1] + ')'
    return temp

In [6]:
join_sym('Not',['K'])

'Not( K)'

In [7]:
jj = Or(hagrid, dumbledore)

In [8]:
jj.evaluate({'rain': True, 'hagrid': False, 'dumbledore': False})

False

In [52]:
di = {}
lists = ['P','B','G','W']
for i in range(0,4):
    for j in range(1,5):
        for k in lists:
            di[f'{k}{i}{j}'] = Symbol(f'{k}{i}{j}')

In [53]:
di

{'P01': P01,
 'B01': B01,
 'G01': G01,
 'W01': W01,
 'P02': P02,
 'B02': B02,
 'G02': G02,
 'W02': W02,
 'P03': P03,
 'B03': B03,
 'G03': G03,
 'W03': W03,
 'P04': P04,
 'B04': B04,
 'G04': G04,
 'W04': W04,
 'P11': P11,
 'B11': B11,
 'G11': G11,
 'W11': W11,
 'P12': P12,
 'B12': B12,
 'G12': G12,
 'W12': W12,
 'P13': P13,
 'B13': B13,
 'G13': G13,
 'W13': W13,
 'P14': P14,
 'B14': B14,
 'G14': G14,
 'W14': W14,
 'P21': P21,
 'B21': B21,
 'G21': G21,
 'W21': W21,
 'P22': P22,
 'B22': B22,
 'G22': G22,
 'W22': W22,
 'P23': P23,
 'B23': B23,
 'G23': G23,
 'W23': W23,
 'P24': P24,
 'B24': B24,
 'G24': G24,
 'W24': W24,
 'P31': P31,
 'B31': B31,
 'G31': G31,
 'W31': W31,
 'P32': P32,
 'B32': B32,
 'G32': G32,
 'W32': W32,
 'P33': P33,
 'B33': B33,
 'G33': G33,
 'W33': W33,
 'P34': P34,
 'B34': B34,
 'G34': G34,
 'W34': W34}

In [11]:
def gen_breeze(neighbor):
    br_rules = ''
    for i in range(1,5):
        for j in range(1,5):
            temp_r = join_sym("Biconditional",[f"di['B{i}{j}']",join_sym('Or',neighbor[i][j],"di['P","']")])
            br_rules += temp_r + ','
    return br_rules

In [12]:
gen_breeze(neighbors)

"Biconditional( di['B11'], Or( di['P12'], di['P21'])),Biconditional( di['B12'], Or( di['P13'], di['P22'], di['P11'])),Biconditional( di['B13'], Or( di['P14'], di['P23'], di['P12'])),Biconditional( di['B14'], Or( di['P24'], di['P13'])),Biconditional( di['B21'], Or( di['P11'], di['P22'], di['P31'])),Biconditional( di['B22'], Or( di['P12'], di['P23'], di['P32'], di['P21'])),Biconditional( di['B23'], Or( di['P13'], di['P24'], di['P33'], di['P22'])),Biconditional( di['B24'], Or( di['P14'], di['P34'], di['P23'])),Biconditional( di['B31'], Or( di['P21'], di['P32'], di['P41'])),Biconditional( di['B32'], Or( di['P22'], di['P33'], di['P42'], di['P31'])),Biconditional( di['B33'], Or( di['P23'], di['P34'], di['P43'], di['P32'])),Biconditional( di['B34'], Or( di['P24'], di['P44'], di['P33'])),Biconditional( di['B41'], Or( di['P31'], di['P42'])),Biconditional( di['B42'], Or( di['P32'], di['P43'], di['P41'])),Biconditional( di['B43'], Or( di['P33'], di['P44'], di['P42'])),Biconditional( di['B44'], Or

In [15]:
knowledge_2 = And(Not(di['P11']),Not(di['B11']),Not(di['B12']),di['B21'],Biconditional( di['B11'], Or( di['P12'], di['P21'])),Biconditional( di['B12'], Or( di['P13'], di['P22'], di['P11'])),Biconditional( di['B13'], Or( di['P14'], di['P23'], di['P12'])),Biconditional( di['B14'], Or( di['P24'], di['P13'])),Biconditional( di['B21'], Or( di['P11'], di['P22'], di['P31'])),Biconditional( di['B22'], Or( di['P12'], di['P23'], di['P32'], di['P21'])),Biconditional( di['B23'], Or( di['P13'], di['P24'], di['P33'], di['P22'])),Biconditional( di['B24'], Or( di['P14'], di['P34'], di['P23'])),Biconditional( di['B31'], Or( di['P21'], di['P32'], di['P41'])),Biconditional( di['B32'], Or( di['P22'], di['P33'], di['P42'], di['P31'])),Biconditional( di['B33'], Or( di['P23'], di['P34'], di['P43'], di['P32'])),Biconditional( di['B34'], Or( di['P24'], di['P44'], di['P33'])),Biconditional( di['B41'], Or( di['P31'], di['P42'])),Biconditional( di['B42'], Or( di['P32'], di['P43'], di['P41'])),Biconditional( di['B43'], Or( di['P33'], di['P44'], di['P42'])),Biconditional( di['B44'], Or( di['P34'], di['P43'])))

In [ ]:
print(model_check(knowledge_2, di['P31']))

ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41': True, 'B11': False, 'P33': True, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': True, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': True}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41': True, 'B11': False, 'P33': True, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': True, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': False}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41': T

ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41': True, 'B11': False, 'P33': False, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': True, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': True}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41': True, 'B11': False, 'P33': False, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': True, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': False}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41':

ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41': False, 'B11': False, 'P33': True, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': True, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': True}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41': False, 'B11': False, 'P33': True, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': True, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': False}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41':

ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41': False, 'B11': False, 'P33': False, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': True, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': True}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41': False, 'B11': False, 'P33': False, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': True, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': False}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': True, 'P22': False, 'P21': False, 'P41

ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': False, 'P22': False, 'P21': False, 'P41': True, 'B11': False, 'P33': True, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': False, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': True}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': False, 'P22': False, 'P21': False, 'P41': True, 'B11': False, 'P33': True, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': False, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': False}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': False, 'P22': False, 'P21': False, 'P4

ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': False, 'P22': False, 'P21': False, 'P41': True, 'B11': False, 'P33': False, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': False, 'P13': False, 'P44': True, 'B43': True, 'B23': False, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': True}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': False, 'P22': False, 'P21': False, 'P41': True, 'B11': False, 'P33': False, 'B22': True, 'P14': True, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': False, 'P13': False, 'P44': True, 'B43': True, 'B23': False, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': False}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': False, 'P22': False, 'P21': False,

ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': False, 'P22': False, 'P21': False, 'P41': False, 'B11': False, 'P33': True, 'B22': True, 'P14': False, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': False, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': True}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': False, 'P22': False, 'P21': False, 'P41': False, 'B11': False, 'P33': True, 'B22': True, 'P14': False, 'P31': True, 'B33': True, 'P23': True, 'B44': True, 'B14': False, 'P13': False, 'P44': True, 'B43': True, 'B23': True, 'B21': True, 'P11': False, 'B42': True, 'P43': True, 'B31': True, 'B12': False, 'B24': True, 'P42': False}
ll
{'B32': True, 'P34': True, 'P32': True, 'B41': True, 'B13': True, 'B34': True, 'P12': False, 'P24': False, 'P22': False, 'P21': False,

In [2]:
knowledge = And()

In [4]:
kb = And()

In [12]:
kb=(Not(di['P11']))

In [39]:
kb = And()

In [43]:
r1 = eval("Not(di['P12'])")

In [44]:
kb.add(r1)

In [31]:
r1.evaluate({'P11':False})

True

In [45]:
kb

And(Not(P11), Not(P12))

In [51]:
kb.evaluate({'P11':False,'P12':False})

True

In [57]:
ii=0
jj = 1

In [58]:
join_sym("Or",[join_sym('Not',[f"di['G{i}{j}']"]),join_sym('Not',[f"di['G{ii}{jj}']"])])

"Or( Not( di['G34']), Not( di['G01']))"

In [59]:
r1 = eval(join_sym("Or",[join_sym('Not',[f"di['G{i}{j}']"]),join_sym('Not',[f"di['G{ii}{jj}']"])]))

In [60]:
kb = And()

In [61]:
kb.add(r1)

In [62]:
kb

And(Or(Not(G34), Not(G01)))

In [67]:
kb.evaluate({'G01':True,'G34':True})

False

In [71]:
eval("( di['G11'])")

G11